In [3]:
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import pandas
def s(t1,t2):
    sleep(randint(1000*t1,1000*t1+1000*t2)/1000)

In [2]:
serv = Service('../chromedriver.exe')
driver = webdriver.Chrome(service=serv)

link = 'https://tarragona.posidoniaport.com/'

driver.get(link)

#entro a històric i hi poso les dates manualment
#1 de gener 2019 00:00 a 31 de desembre 2019 23:59
#màxim (30) de files per pàgina

In [177]:
#creo dataframe buit
df = pandas.DataFrame(columns = ['Nom', 'Tipus', 'MO', 'Lloyds', 'Bandera', 
                                 'Calat real', 'GT', 'Mànega', 'Eslora'])

In [190]:
#inici loop
pagines=range(0,125)
k=0
grid = 'grid_1666900370744' #aquest canvia a cada sessió nova, cal buscar-lo amb inspeccionar
nextpage = 'next_pager858' #aquest també

#per buscar-los suggereixo fer cntrl+f i buscar grid_ i next_pager, amb això es trobarà ràpidament l'identificador

for pag in pagines:
    k += 1
    print(k,end=' ')    
    
    s(1,2)
    
    table_id = driver.find_element(By.XPATH, "//table[@id='"+grid+"']") 
    rows = table_id.find_elements(By.CSS_SELECTOR, "[tabindex='-1']")
    
    for row in rows:
        nom = row.find_elements(By.CSS_SELECTOR,"[aria-describedby="+grid+"_nombuq]")[0].text

        if nom not in df['Nom'].values:
            row.find_elements(By.CLASS_NAME,"linkformgisweb")[0].click()
            s(5,3)
            
            df_tmp = pandas.DataFrame({
                'Nom':nom,
                'Tipus':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: destipbuq']")[0].get_attribute('value'),
                'MO':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: codbuq']")[0].get_attribute('value'),
                'Lloyds':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: lloyds']")[0].get_attribute('value'),
                'Bandera':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: bandera']")[0].get_attribute('value'),
                'Calat real':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: calrea']")[0].get_attribute('value'),
                'GT':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: gt']")[0].get_attribute('value'),
                'Mànega':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: manga']")[0].get_attribute('value'),
                'Eslora':driver.find_elements(By.CSS_SELECTOR,"[data-bind='value: eslora']")[0].get_attribute('value')},
                index=[0])
            
            df_tmp['GT']=df_tmp['GT'].str.replace('.','', regex=True)
            
            df=pandas.concat([df,df_tmp])
            
            driver.find_elements(By.CSS_SELECTOR,"[class='ui-icon ui-icon-closethick']")[1].click()
            s(1,1)
    
    #canvi de pàgina    
    driver.find_element(By.XPATH, "//td[@id='"+nextpage+"']").click() #també cal actualitzar-lo a cada nova sessió

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 

In [203]:
df2 = df.astype({"Calat real":"float",
                "GT":"float",
                "Mànega":"float",
                "Eslora":"float"})

In [204]:
df2.index = range(1,len(df2)+1)
df2.to_excel('../dataset/Característiques Vaixells_2019.xlsx')